In [1]:
import pandas as pd
import numpy as np
import os
import PIL
from pathlib import Path
from PIL import Image
# import scipy
import IPython
from IPython.display import display
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Convolution2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical
import tensorflow as tf
        

2022-09-02 22:11:51.876991: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:/usr/lib/cuda/include:/usr/lib/cuda/lib64:/usr/lib/cuda/include:/usr/lib/cuda/lib64:/usr/lib/cuda/include:/usr/lib/cuda/lib64:
2022-09-02 22:11:51.877015: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
root_path = Path().resolve()

In [3]:
def image_to_pixel(img,maxPixel=32):
    imScalled = img.resize((maxPixel,maxPixel))
    img_pixel = list(imScalled.getdata())
    return img_pixel

#reading all images
train_path = os.path.join(root_path, 'data/training')
validate_path = os.path.join(root_path, 'data/validation')
train_image_df=pd.DataFrame(columns=['category','file_path'])
for folder in os.listdir(train_path):
    for fileNames in os.listdir(train_path+'/'+folder):
        row={'category':folder,'file_path':train_path+'/'+folder+'/'+fileNames}
        train_image_df.loc[len(train_image_df.index)] = [folder,train_path+'/'+folder+'/'+fileNames]
        
        #IPython.display.Image(data_path+'/'+folder+'/'+fileNames)

valid_image_df=pd.DataFrame(columns=['category','file_path'])
for folder in os.listdir(validate_path):
    for fileNames in os.listdir(validate_path+'/'+folder):
        row={'category':folder,'file_path':validate_path+'/'+folder+'/'+fileNames}
        valid_image_df.loc[len(valid_image_df.index)] = [folder,validate_path+'/'+folder+'/'+fileNames]
        
        #IPython.display.Image(data_path+'/'+folder+'/'+fileNames)



In [4]:
# valid_image_df.head(1)
img = Image.open(f'{root_path}/data/training/aeroplane/2007_000033.jpg')
print(img.size)
# img
maxPixel = 128
imScalled = img.resize((maxPixel,maxPixel))
img_pixel = np.reshape(list(imScalled.getdata()),(maxPixel,maxPixel,3))
img_pixel
print(img_pixel.size)

(500, 366)
49152


In [12]:
maxPixel = 128
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
valid_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(os.path.join(root_path, 'data/training'),  batch_size=200, class_mode = 'binary', target_size = (maxPixel, maxPixel),classes=['aeroplane','others'])
valid_generator = valid_datagen.flow_from_directory(os.path.join(root_path, 'data/validation'),  batch_size=200, class_mode = 'binary', target_size = (maxPixel, maxPixel),classes=['aeroplane','others'])

base_model = InceptionV3(input_shape = (maxPixel, maxPixel, 3), include_top = False, weights = 'imagenet', classes=20)

for layer in base_model.layers:
    layer.trainable = False


Found 83 images belonging to 2 classes.
Found 723 images belonging to 2 classes.


In [91]:
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

/home/aumaron/Desktop/neural_networks/neural_env/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [97]:
Flatten()(base_model.output)

<KerasTensor: shape=(None, 8192) dtype=float32 (created by layer 'flatten_5')>

In [27]:
# Fit the model
plane_detector = model.fit_generator(
    train_generator, 
    validation_data=valid_generator, 
    steps_per_epoch=1,
#     validation_steps=valid_generator.samples/valid_generator.batch_size,
    epochs=10
)

Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 0.3653 - acc: 0.8675 - val_loss: 1.1856 - val_acc: 0.5311
Epoch 2/10
1/1 [==============================] - 6s 6s/step - loss: 0.5657 - acc: 0.7831 - val_loss: 0.0430 - val_acc: 0.9889
Epoch 3/10
1/1 [==============================] - 6s 6s/step - loss: 1.1991 - acc: 0.6988 - val_loss: 2.2805 - val_acc: 0.2351
Epoch 4/10
1/1 [==============================] - 7s 7s/step - loss: 1.2606 - acc: 0.5904 - val_loss: 0.0634 - val_acc: 0.9834
Epoch 5/10
1/1 [==============================] - 7s 7s/step - loss: 1.3988 - acc: 0.6867 - val_loss: 0.3349 - val_acc: 0.8714
Epoch 6/10
1/1 [==============================] - 7s 7s/step - loss: 0.1335 - acc: 0.9518 - val_loss: 0.1816 - val_acc: 0.9350
Epoch 7/10
1/1 [==============================] - 6s 6s/step - loss: 0.1028 - acc: 0.9518 - val_loss: 0.1874 - val_acc: 0.9295
Epoch 8/10
1/1 [==============================] - 6s 6s/step - loss: 0.1222 - acc: 0.9639 - val_loss: 0.1446 - 

In [94]:
model.output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_7')>

In [63]:
features = np.zeros(shape=(83, 2, 2, 2048))  # Must be equal to the output of the convolutional base
labels = np.zeros(shape=(83))

# Pass data through convolutional base
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = base_model.predict(inputs_batch)
    features[i * 200: (i + 1) * 200] = features_batch
    labels[i * 200: (i + 1) * 200] = labels_batch
    i += 1
    if i * 200 >= 83:
        break

In [76]:
for i in range(83):
    features[i]
features.shape

(83, 2, 2, 2048)

In [88]:
features[0].shape

(2, 2, 2048)

In [54]:
for inputs_batch, labels_batch in train_generator:
    print(inputs_batch.shape)
#     print(model.predict(inputs_batch).shape)

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

KeyboardInterrupt: 